In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/nhj4247/python_cache/'

In [2]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)
fill_mask("Send these <mask> back!")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'sequence': 'Send these pictures back!',
  'score': 0.16661524772644043,
  'token': 3493,
  'token_str': ' pictures'},
 {'sequence': 'Send these photos back!',
  'score': 0.10792797058820724,
  'token': 2356,
  'token_str': ' photos'},
 {'sequence': 'Send these emails back!',
  'score': 0.0767090767621994,
  'token': 5575,
  'token_str': ' emails'},
 {'sequence': 'Send these images back!',
  'score': 0.0486077181994915,
  'token': 3156,
  'token_str': ' images'},
 {'sequence': 'Send these letters back!',
  'score': 0.04841756820678711,
  'token': 5430,
  'token_str': ' letters'}]

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/scratch/nhj4247/data/tweeteval/datasets/hate/train_text.txt",
    block_size=512,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    output_dir="./roberta-retrained",
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=48,
    save_steps=500,
    save_total_limit=2,
    seed=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

trainer.train()

trainer.save_model("./roberta-retrained")



***** Running training *****
  Num examples = 8993
  Num Epochs = 25
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 4700


Step,Training Loss
500,2.340100


Saving model checkpoint to ./roberta-retrained/checkpoint-500
Configuration saved in ./roberta-retrained/checkpoint-500/config.json
Model weights saved in ./roberta-retrained/checkpoint-500/pytorch_model.bin
